# Tasks Latencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd

In [ ]:
rendert_df = pd.read_csv('./data.csv')
sf_df = pd.read_csv('./data_sf.csv')

rendert_df.set_index('Time', inplace=True)
sf_df.set_index('Time', inplace=True)

In [ ]:
def get_events(task_df, task_name):
    wakeup = task_df[(task_df.curr_state == 'W') & (task_df.next_state == 'A')]
    running = task_df[(task_df.curr_state == 'A')]
    preempt = task_df[(task_df.curr_state.isin(['R', 'R+'])) & (task_df.next_state == 'A')]
    suspend = task_df[(task_df.curr_state == 'D') & (task_df.next_state == 'W')]
    
    t_end = wakeup['t_start'] + wakeup['t_delta']
    w_event = [[start, end, task_name] for start, end in t_end.iteritems()]
    t_end = running['t_start'] + running['t_delta']
    r_event = [[start, end, task_name] for start, end in t_end.iteritems()]
    t_end = preempt['t_start'] + preempt['t_delta']
    p_event = [[start, end, task_name] for start, end in t_end.iteritems()]
    t_end = suspend['t_start'] + suspend['t_delta']
    u_event = [[start, end, task_name] for start, end in t_end.iteritems()]
    
    return r_event, w_event, p_event, u_event

In [ ]:
rt_events = get_events(rendert_df, "RenderThread")
sf_events = get_events(sf_df, "SurfaceFlinger")

In [ ]:
EVENTS = {}
EVENTS["Running"] = rt_events[0] + sf_events[0]
EVENTS["WakeupLat"] = rt_events[1] + sf_events[1]
EVENTS["PreemptLat"] = rt_events[2] + sf_events[2]
EVENTS["SuspendedUnint"] = rt_events[3] + sf_events[3]

x_max = max(sf_df.index.max(), rendert_df.index.max())

trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,
                 lanes=["RenderThread", "SurfaceFlinger"],
                 domain=[0,x_max]
                ).view()

In [ ]:
EVENTS = {}
EVENTS["Running"] = rt_events[0] + sf_events[0]
EVENTS["WakeupLat"] = rt_events[1] + sf_events[1]
EVENTS["PreemptLat"] = rt_events[2] + sf_events[2]
EVENTS["SuspendedUnint"] = rt_events[3] + sf_events[3]

x_max = max(sf_df.index.max(), rendert_df.index.max())

trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,
                 lanes=["RenderThread", "SurfaceFlinger"],
                 domain=[0,x_max],
                 color_map={"Running" : "green", "WakeupLat" : "purple", "PreemptLat" : "orange", "SuspendedUnint" : "red"}
                ).view()